<a href="https://colab.research.google.com/github/ARADHYA299/OpensourceLLMprojects/blob/main/Audio_%3ETextGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q openai-whisper transformers google-generativeai deepseek openai numpy bitsandbytes

In [3]:
import whisper
import google.generativeai as genai
from transformers import AutoTokenizer, AutoModelForCausalLM , BitsAndBytesConfig , TextStreamer
import torch
import numpy as np
import openai

In [4]:
Audio_model  = "whisper-1"
Falcon_model = "tiiuae/falcon-7b-instruct"

In [5]:
from google.colab import drive
drive.mount("/content/drive")
audio_filename = "/content/drive/My Drive/Colab Notebooks/denver_extract.mp3"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from getpass import getpass
from huggingface_hub import login

hf_token = getpass("my-hf-token")

login(hf_token, add_to_git_credential=True)

my-hf-token··········


In [7]:
model = whisper.load_model("base")

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [8]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

In [9]:
transcription = model.transcribe(audio_filename)
print(transcription)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': " kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason the back of the logo is considered water. So let you see the creation of the logo here. And yes, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of indigenous peoples day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th, please rise with the pledge of allegiance by Councilman Lopez. One makes me under God in the visibl

In [10]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_message = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"
messages = [{"role":"system" , "content":system_message} , {"role":"user" , "content":user_message}]

In [14]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(Falcon_model)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(conversation = messages , return_tensors="pt" , padding = True)
attention_mask = inputs.ne(tokenizer.pad_token_id)
streamer = TextStreamer(tokenizer)
model1 = AutoModelForCausalLM.from_pretrained(Falcon_model , device_map = "auto" )
outputs = model1.generate(inputs , max_new_tokens = 1024 , streamer = streamer)

Token indices sequence length is longer than the specified maximum sequence length for this model (29660 > 2048). Running this sequence through the model will result in indexing errors


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]